# Can We Predict Churn Rate?: The Lifeline of Startups and Their Customers: Data Integration

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Loading and initial exploration of datasets
df_members = pd.read_csv('data/members_v3.csv')
df_train = pd.read_csv('data/train_v2.csv')
df_trans = pd.read_csv('data/transactions_v2.csv')
df_userlog = pd.read_csv('data/user_logs_v2.csv')
df_test = pd.read_csv('data/sample_submission_v2.csv')

# Create a list of dataframes
dataframes = [df_members, df_train, df_trans, df_userlog, df_test]

## 3: Data Cleaning and Preparation

## Preparation for Data Merger

In this dataset, there are multiple entries for the 'msno' (ID). Create a refined 'df_trans2' DataFrame that retains the latest 'membership_expire_date' row for each 'msno', after making a copy of 'df_members' and converting the date column. Also, calculate the total count of duplicated 'msno' values.

In [3]:
# Create a copy of df_members
df_trans2 = df_trans.copy()

# Convert the 'membership_expire_date' column to datetime format
df_trans2['membership_expire_date'] = pd.to_datetime(df_trans2['membership_expire_date'], format='%Y%m%d')

# Group by 'msno' and get the row with the latest 'membership_expire_date' for each 'msno'
df_trans2 = df_trans2.sort_values('membership_expire_date').drop_duplicates('msno', keep='last')

df_trans2['msno'].duplicated().sum()

0

Perform data aggregation on the 'df_userlog' dataset by copying it, dropping the date column, counting 'msno' occurrences, summing other columns after grouping by 'msno', and merging the results with occurrence counts. The final aggregated DataFrame 'df_userlog_sum' is then displayed.

In [4]:
# Copy the original data
df_userlog_copy = df_userlog.copy()

# Drop the date column
df_userlog.drop(columns=["date"], inplace=True)

# Count the occurrences of msno
msno_counts = df_userlog.groupby("msno").size().reset_index(name='msno_count')

# Group the data by msno and sum the other columns
df_userlog_sum = df_userlog.groupby("msno").sum().reset_index()

# Merge the summed dataframe with the count of msno occurrences
df_userlog_sum = df_userlog_sum.merge(msno_counts, on="msno", how="left")

# Rename the columns
for column in df_userlog_sum.columns:
    if column != "msno" and column != "msno_count":
        new_column_name = f"{column}_sum"
        df_userlog_sum.rename(columns={column: new_column_name}, inplace=True)

print(df_userlog_sum)


                                                 msno  num_25_sum  num_50_sum  \
0        +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=          86          11   
1        +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=         191          90   
2        +++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=          43          12   
3        +++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=         207         163   
4        ++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=         105          24   
...                                               ...         ...         ...   
1103889  zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=         136          69   
1103890  zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=          17           8   
1103891  zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=          33          10   
1103892  zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=          10           1   
1103893  zzzueVTwIa5TjXnG2c77bohCVkuksqLkd5mQTP0wTwQ=           2           0   

         num_75_sum  num_98

## Data Integration

### Train Dataset

Combining data from different sources for analysis

In [5]:
# merging members data with train data
df_train_members = pd.merge(df_train, df_members, on='msno', how='left')
# merging transactions data with train members data
df_train_mem_trans = pd.merge(df_train_members, df_trans2, on='msno', how='left')
# merging user logs data with train members transaction data
train_dataset = pd.merge(df_train_mem_trans, df_userlog_sum, on='msno', how='left')

train_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,membership_expire_date,is_cancel,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,msno_count
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,...,NaT,NaN,186.0,23.0,13.0,10.0,318.0,348.0,80598.557,11.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0,36.0,30.0,180.0,...,2017-04-11,0.0,0.0,4.0,2.0,0.0,26.0,30.0,6986.509,6.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0,15.0,90.0,300.0,...,2017-06-15,0.0,239.0,57.0,32.0,22.0,205.0,432.0,67810.467,20.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0,41.0,30.0,149.0,...,2018-01-08,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0,41.0,30.0,99.0,...,2017-09-18,0.0,9.0,7.0,4.0,4.0,962.0,548.0,239882.241,15.0


In [6]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 970960 entries, 0 to 970959
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   msno                    970960 non-null  object        
 1   is_churn                970960 non-null  int64         
 2   city                    860967 non-null  float64       
 3   bd                      860967 non-null  float64       
 4   gender                  388905 non-null  object        
 5   registered_via          860967 non-null  float64       
 6   registration_init_time  860967 non-null  float64       
 7   payment_method_id       933578 non-null  float64       
 8   payment_plan_days       933578 non-null  float64       
 9   plan_list_price         933578 non-null  float64       
 10  actual_amount_paid      933578 non-null  float64       
 11  is_auto_renew           933578 non-null  float64       
 12  transaction_date        933578

In [7]:
train_dataset.describe()

,is_churn,city,bd,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,is_cancel,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum,msno_count
count,970960.000000,860967.000000,860967.000000,860967.000000,8.609670e+05,933578.000000,933578.000000,933578.000000,933578.000000,933578.000000,9.335780e+05,933578.000000,754551.000000,754551.000000,754551.000000,754551.000000,754551.000000,754551.000000,7.545510e+05,754551.000000
mean,0.089942,5.937295,13.583854,6.892744,2.013265e+07,38.893771,33.898736,145.078097,144.834934,0.911185,2.017018e+07,0.012890,108.773786,26.257260,16.545723,18.713420,535.902252,511.878935,1.397844e+05,17.935095
std,0.286099,6.441203,20.064040,1.937794,3.011174e+04,3.795252,33.297362,140.390246,140.483812,0.284477,1.512955e+03,0.112801,179.003032,38.707599,22.399375,41.691069,763.731803,594.729644,1.855241e+05,9.676949
min,0.000000,1.000000,-3152.000000,3.000000,2.004033e+07,3.000000,0.000000,0.000000,0.000000,0.000000,2.015010e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.500000e-02,1.000000
25%,0.000000,1.000000,0.000000,7.000000,2.012021e+07,38.000000,30.000000,99.000000,99.000000,1.000000,2.017031e+07,0.000000,19.000000,5.000000,3.000000,3.000000,108.000000,124.000000,3.020290e+04,9.000000
50%,0.000000,1.000000,0.000000,7.000000,2.014060e+07,41.000000,30.000000,149.000000,149.000000,1.000000,2.017032e+07,0.000000,55.000000,14.000000,10.000000,10.000000,305.000000,330.000000,8.264024e+04,19.000000
75%,0.000000,13.000000,27.000000,9.000000,2.016012e+07,41.000000,30.000000,149.000000,149.000000,1.000000,2.017033e+07,0.000000,133.000000,33.000000,22.000000,23.000000,660.000000,680.000000,1.757171e+05,27.000000
max,1.000000,22.000000,2016.000000,13.000000,2.017042e+07,41.000000,450.000000,2000.000000,2000.000000,1.000000,2.017033e+07,1.000000,46820.000000,3163.000000,1310.000000,15164.000000,57675.000000,23403.000000,1.433739e+07,31.000000


Performing a missing value analysis on the 'train_dataset'. We'll address handling the null values in subsequent steps.

In [8]:
# Calculate the number of missing values for each feature
missing_values = train_dataset.isnull().sum()

# Calculate the percentage of missing values for each feature
missing_percentage = (train_dataset.isnull().sum() / len(train_dataset)) * 100

# Display the results as a DataFrame
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_df.sort_values(by='Percentage', ascending=False)

,Missing Values,Percentage
gender,582055,59.946342
msno_count,216409,22.288148
total_secs_sum,216409,22.288148
num_unq_sum,216409,22.288148
num_100_sum,216409,22.288148
num_985_sum,216409,22.288148
num_75_sum,216409,22.288148
num_50_sum,216409,22.288148
num_25_sum,216409,22.288148
city,109993,11.328273


In [9]:
# Check duplication of msno
num_duplicated = train_dataset['msno'].duplicated().sum()
print(f"Number of duplicated msno: {num_duplicated}")

Number of duplicated msno: 0


## Checkpoint

In [10]:
# Save the randomly picked rows to new CSV files
train_dataset.to_csv('/Users/halmorishima/Github/mycaprepo/data/train_dataset.csv', index=False)